In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
import numpy as np
from scipy.stats import norm
import math
import numpy as np
import math
import importlib 
from scipy.stats import norm
import scipy.integrate as nInt
from scipy.stats import t as myT
import numpy.linalg as anp
import scipy
from Functions import *

In [2]:
data = pd.read_excel('.\\adatok.xlsx', sheet_name='Sheet3')
states = pd.read_excel('.\\States.xlsx', sheet_name='Sheet1')
data['Hectic'] = states['Hectic_Assignment']
data

,DATUM,NO_OF_FIRMS_TOTAL,NO_OF_DEFAULTS,LGD1,LGD2,LGD3,LGD4,LGD5,P(D|G),Hectic_m,Hectic
0,1996-03-31,5231,1,NaN,NaN,NaN,NaN,NaN,0.000191,0,0
1,1996-06-30,5231,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,1996-09-30,5231,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,1996-12-31,5231,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,1997-03-31,6157,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...
111,2023-12-31,66385,70,0.306554,0.185995,0.199405,0.300555,0.497117,0.001054,0,1
112,2024-03-31,62836,58,0.426458,0.276759,0.278882,0.369540,0.590945,0.000923,0,1
113,2024-06-30,62836,71,0.448654,0.300499,0.315679,0.406456,0.592471,0.001130,0,1
114,2024-09-30,62836,96,0.337895,0.172183,0.172353,0.306770,0.498617,0.001528,0,1


In [3]:
states

,DATUM,Mean_Quiet,Mean_Hectic,Quiet_Assignment,Hectic_Assignment
0,1996Q1,0.522362,0.477638,1,0
1,1996Q2,0.642279,0.357721,1,0
2,1996Q3,0.606472,0.393528,1,0
3,1996Q4,0.608573,0.391427,1,0
4,1997Q1,0.624316,0.375684,1,0
...,...,...,...,...,...
111,2023Q4,0.356317,0.643683,0,1
112,2024Q1,0.202409,0.797591,0,1
113,2024Q2,0.295976,0.704024,0,1
114,2024Q3,0.380533,0.619467,0,1


In [4]:
p_mean = data['P(D|G)'].mean()
p_mean

np.float64(0.0011946195896439494)

## Calculating a correalation estimation according to the Basel article

In [5]:
PD = p_mean

In [6]:
R = calculate_R(PD)

# One factor Gaussian

In [7]:
#M - number of simulations
#N - number of assets in the portfolio
M = 100
alpha = np.array([0.95,0.97,0.99,0.995,0.999])
rho = R

### Simulating the expected number of defaults

In [8]:
# Untitled.ipynb


In [9]:
#data.to_excel('output.xlsx', index=False)

### Simulating the risk measures

In [10]:
data['varSim'] = None
data['esSim'] = None
data['varSim'] = data['varSim'].astype(object)
data['esSim'] = data['esSim'].astype(object)
for index, row in data.iterrows():
    N = row['NO_OF_FIRMS_TOTAL']
    p = p_mean
    c = np.ones((N,1))
    pdf, cdf, varSim, esSim = oneFactorGaussianModel(N,M,p,c,rho,alpha)
    data.at[index, 'varSim'] = np.nan_to_num(varSim).tolist() if isinstance(varSim, np.ndarray) else varSim
    data.at[index, 'esSim'] = np.nan_to_num(esSim).tolist() if isinstance(esSim, np.ndarray) else esSim

In [11]:
data

,DATUM,NO_OF_FIRMS_TOTAL,NO_OF_DEFAULTS,LGD1,LGD2,LGD3,LGD4,LGD5,P(D|G),Hectic_m,Hectic,varSim,esSim
0,1996-03-31,5231,1,NaN,NaN,NaN,NaN,NaN,0.000191,0,0,"[37.0, 44.0, 138.0, 138.0, 138.0]","[61.6, 76.0, 138.0, 138.0, 138.0]"
1,1996-06-30,5231,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,"[25.0, 47.0, 57.0, 57.0, 57.0]","[43.8, 53.0, 57.0, 57.0, 57.0]"
2,1996-09-30,5231,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,"[18.0, 22.0, 43.0, 43.0, 43.0]","[25.8, 30.0, 43.0, 43.0, 43.0]"
3,1996-12-31,5231,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,"[34.0, 40.0, 91.0, 91.0, 91.0]","[49.6, 58.333333333333336, 91.0, 91.0, 91.0]"
4,1997-03-31,6157,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,"[41.0, 59.0, 70.0, 70.0, 70.0]","[56.4, 64.0, 70.0, 70.0, 70.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,2023-12-31,66385,70,0.306554,0.185995,0.199405,0.300555,0.497117,0.001054,0,1,"[206.0, 495.0, 693.0, 693.0, 693.0]","[443.6, 599.3333333333334, 693.0, 693.0, 693.0]"
112,2024-03-31,62836,58,0.426458,0.276759,0.278882,0.369540,0.590945,0.000923,0,1,"[469.0, 497.0, 654.0, 654.0, 654.0]","[538.2, 582.6666666666666, 654.0, 654.0, 654.0]"
113,2024-06-30,62836,71,0.448654,0.300499,0.315679,0.406456,0.592471,0.001130,0,1,"[439.0, 1038.0, 1961.0, 1961.0, 1961.0]","[1043.2, 1405.3333333333333, 1961.0, 1961.0, 1..."
114,2024-09-30,62836,96,0.337895,0.172183,0.172353,0.306770,0.498617,0.001528,0,1,"[323.0, 420.0, 610.0, 610.0, 610.0]","[456.0, 540.6666666666666, 610.0, 610.0, 610.0]"


## Analytical one-factor Gaussian

In [12]:
p = p_mean

data['varAnalytic'] = None
data['esAnalytic'] = None
data['varAnalytic'] = data['varAnalytic'].astype(object)
data['esAnalytic'] = data['esAnalytic'].astype(object)
for index, row in data.iterrows():
    N = row['NO_OF_FIRMS_TOTAL']
    c = np.ones((N,1))
    rho = 0.02
    pdf, cdf, varAnalytic, esAnalytic = asrfModel(p, rho, c, alpha)
    data.at[index, 'varAnalytic'] = np.nan_to_num(varAnalytic).tolist() if isinstance(varAnalytic, np.ndarray) else varAnalytic
    data.at[index, 'esAnalytic'] = np.nan_to_num(esAnalytic).tolist() if isinstance(esAnalytic, np.ndarray) else esAnalytic

C:\Users\beben\OneDrive\Documents\Egyetem\Temalabor1\Codes\model_functions.py:89: RuntimeWarning: invalid value encountered in divide
  prob /= np.sum(prob)


In [13]:
data

,DATUM,NO_OF_FIRMS_TOTAL,NO_OF_DEFAULTS,LGD1,LGD2,LGD3,LGD4,LGD5,P(D|G),Hectic_m,Hectic,varSim,esSim,varAnalytic,esAnalytic
0,1996-03-31,5231,1,NaN,NaN,NaN,NaN,NaN,0.000191,0,0,"[37.0, 44.0, 138.0, 138.0, 138.0]","[61.6, 76.0, 138.0, 138.0, 138.0]","[50.7095225131853, 51.766081647171724, 52.8226...","[0.0, 0.0, 0.0, 0.0, 0.0]"
1,1996-06-30,5231,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,"[25.0, 47.0, 57.0, 57.0, 57.0]","[43.8, 53.0, 57.0, 57.0, 57.0]","[50.7095225131853, 51.766081647171724, 52.8226...","[0.0, 0.0, 0.0, 0.0, 0.0]"
2,1996-09-30,5231,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,"[18.0, 22.0, 43.0, 43.0, 43.0]","[25.8, 30.0, 43.0, 43.0, 43.0]","[50.7095225131853, 51.766081647171724, 52.8226...","[0.0, 0.0, 0.0, 0.0, 0.0]"
3,1996-12-31,5231,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,"[34.0, 40.0, 91.0, 91.0, 91.0]","[49.6, 58.333333333333336, 91.0, 91.0, 91.0]","[50.7095225131853, 51.766081647171724, 52.8226...","[0.0, 0.0, 0.0, 0.0, 0.0]"
4,1997-03-31,6157,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,"[41.0, 59.0, 70.0, 70.0, 70.0]","[56.4, 64.0, 70.0, 70.0, 70.0]","[59.6862034245234, 60.92979634900331, 62.17338...","[0.0, 0.0, 0.0, 0.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,2023-12-31,66385,70,0.306554,0.185995,0.199405,0.300555,0.497117,0.001054,0,1,"[206.0, 495.0, 693.0, 693.0, 693.0]","[443.6, 599.3333333333334, 693.0, 693.0, 693.0]","[643.5388361762199, 656.9473007355181, 670.355...","[0.0, 0.0, 0.0, 0.0, 0.0]"
112,2024-03-31,62836,58,0.426458,0.276759,0.278882,0.369540,0.590945,0.000923,0,1,"[469.0, 497.0, 654.0, 654.0, 654.0]","[538.2, 582.6666666666666, 654.0, 654.0, 654.0]","[609.1346887093312, 621.826325058628, 634.5179...","[0.0, 0.0, 0.0, 0.0, 0.0]"
113,2024-06-30,62836,71,0.448654,0.300499,0.315679,0.406456,0.592471,0.001130,0,1,"[439.0, 1038.0, 1961.0, 1961.0, 1961.0]","[1043.2, 1405.3333333333333, 1961.0, 1961.0, 1...","[609.1346887093312, 621.826325058628, 634.5179...","[0.0, 0.0, 0.0, 0.0, 0.0]"
114,2024-09-30,62836,96,0.337895,0.172183,0.172353,0.306770,0.498617,0.001528,0,1,"[323.0, 420.0, 610.0, 610.0, 610.0]","[456.0, 540.6666666666666, 610.0, 610.0, 610.0]","[609.1346887093312, 621.826325058628, 634.5179...","[0.0, 0.0, 0.0, 0.0, 0.0]"


# Results of the Gaussian model

In [14]:
results = results = data[['DATUM', 'NO_OF_FIRMS_TOTAL', 'varSim', 'esSim', 'varAnalytic', 'esAnalytic' ]]
alpha = np.array([0.95,0.97,0.99,0.995,0.999])
results

,DATUM,NO_OF_FIRMS_TOTAL,varSim,esSim,varAnalytic,esAnalytic
0,1996-03-31,5231,"[37.0, 44.0, 138.0, 138.0, 138.0]","[61.6, 76.0, 138.0, 138.0, 138.0]","[50.7095225131853, 51.766081647171724, 52.8226...","[0.0, 0.0, 0.0, 0.0, 0.0]"
1,1996-06-30,5231,"[25.0, 47.0, 57.0, 57.0, 57.0]","[43.8, 53.0, 57.0, 57.0, 57.0]","[50.7095225131853, 51.766081647171724, 52.8226...","[0.0, 0.0, 0.0, 0.0, 0.0]"
2,1996-09-30,5231,"[18.0, 22.0, 43.0, 43.0, 43.0]","[25.8, 30.0, 43.0, 43.0, 43.0]","[50.7095225131853, 51.766081647171724, 52.8226...","[0.0, 0.0, 0.0, 0.0, 0.0]"
3,1996-12-31,5231,"[34.0, 40.0, 91.0, 91.0, 91.0]","[49.6, 58.333333333333336, 91.0, 91.0, 91.0]","[50.7095225131853, 51.766081647171724, 52.8226...","[0.0, 0.0, 0.0, 0.0, 0.0]"
4,1997-03-31,6157,"[41.0, 59.0, 70.0, 70.0, 70.0]","[56.4, 64.0, 70.0, 70.0, 70.0]","[59.6862034245234, 60.92979634900331, 62.17338...","[0.0, 0.0, 0.0, 0.0, 0.0]"
...,...,...,...,...,...,...
111,2023-12-31,66385,"[206.0, 495.0, 693.0, 693.0, 693.0]","[443.6, 599.3333333333334, 693.0, 693.0, 693.0]","[643.5388361762199, 656.9473007355181, 670.355...","[0.0, 0.0, 0.0, 0.0, 0.0]"
112,2024-03-31,62836,"[469.0, 497.0, 654.0, 654.0, 654.0]","[538.2, 582.6666666666666, 654.0, 654.0, 654.0]","[609.1346887093312, 621.826325058628, 634.5179...","[0.0, 0.0, 0.0, 0.0, 0.0]"
113,2024-06-30,62836,"[439.0, 1038.0, 1961.0, 1961.0, 1961.0]","[1043.2, 1405.3333333333333, 1961.0, 1961.0, 1...","[609.1346887093312, 621.826325058628, 634.5179...","[0.0, 0.0, 0.0, 0.0, 0.0]"
114,2024-09-30,62836,"[323.0, 420.0, 610.0, 610.0, 610.0]","[456.0, 540.6666666666666, 610.0, 610.0, 610.0]","[609.1346887093312, 621.826325058628, 634.5179...","[0.0, 0.0, 0.0, 0.0, 0.0]"


In [15]:
date_to_plot = '1996-03-31'

data['DATUM'] = pd.to_datetime(data['DATUM'])

if date_to_plot in data['DATUM'].dt.strftime('%Y-%m-%d').values:
    row = data.loc[data['DATUM'] == date_to_plot].iloc[0]
    varSim_values = row['varSim'] if isinstance(row['varSim'], list) else []
    esSim_values = row['esSim'] if isinstance(row['esSim'], list) else []
    varAnalytic_values = row['varAnalytic'] if isinstance(row['varAnalytic'], list) else []
    esAnalytic_values = row['esAnalytic'] if isinstance(row['esAnalytic'], list) else []

    if len(varSim_values) == len(alpha) and len(esSim_values) == len(alpha) and len(varAnalytic_values) == len(alpha) and len(esAnalytic_values) == len(alpha):
        risk_measures_df = pd.DataFrame({
            'Percentile': alpha,
            'varSim': varSim_values,
            'esSim': esSim_values,
            'varAnalytic': varAnalytic_values,
            'esAnalytic': esAnalytic_values,
        })

        print(risk_measures_df)
    else:
        print(f"Risk measures for the date {date_to_plot} are incomplete or not matching the expected length.")
else:
    print(f"No data available for the date {date_to_plot}.")



   Percentile  varSim  esSim  varAnalytic  esAnalytic
0       0.950    37.0   61.6    50.709523         0.0
1       0.970    44.0   76.0    51.766082         0.0
2       0.990   138.0  138.0    52.822641         0.0
3       0.995   138.0  138.0    53.086781         0.0
4       0.999   138.0  138.0    53.298092         0.0


# State dependent model

To get the state dependent models, we need further parameters:<br>
    -$\omega$<br>
    -$\alpha^H$: the correlation in hectic states<br>
    -$\alpha^Q$: the correlation in quiet states<br>
    -P(Q|S) & P(H|S)<br>


In [16]:
p_quiet_m = data.loc[data['Hectic_m'] == 0, 'P(D|G)'].mean()
p_hectic_m = data.loc[data['Hectic_m'] == 1, 'P(D|G)'].mean()
p_quiet = data.loc[data['Hectic'] == 0, 'P(D|G)'].mean()
p_hectic = data.loc[data['Hectic_m'] == 1, 'P(D|G)'].mean()
print(p_quiet_m, p_hectic_m ,p_quiet, p_hectic)

0.0006576842084796814 0.0024149272741081935 0.0002503831932652927 0.0024149272741081935


In [19]:
R_quiet_m = calculate_R(p_quiet_m)
R_hectic_m = calculate_R(p_hectic_m)
R_quiet = calculate_R(p_quiet)
R_hectic = calculate_R(p_hectic)
print(R_quiet_m, R_hectic_m ,R_quiet, R_hectic)

0.15704171577062864 0.14946363266152687 0.15886573375078664 0.14946363266152687
